In [1]:
# Load necessary library
library(readr)

# Define the file path
file_path <- '/content/Final.csv'

# Read the CSV file into a dataframe
dataset <- read_csv(file_path)

# Display the dataset
print(dataset)


Rows: 48284 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): tweet_text, processed_tweet
dbl (7): age, ethnicity, gender, religion, other_cyberbullying, not_cyberbul...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


# A tibble: 48,284 × 9
   tweet_text                  age ethnicity gender religion other_cyberbullying
   <chr>                     <dbl>     <dbl>  <dbl>    <dbl>               <dbl>
 1 In other words #katandan…     0         0      0        0                   0
 2 Why is #aussietv so whit…     0         0      0        0                   0
 3 @XochitlSuckkks a classy…     0         0      0        0                   0
 4 @Jason_Gio meh. :P  than…     0         0      0        0                   0
 5 @RudhoeEnglish This is a…     0         0      0        0                   0
 6 @Raja5aab @Quickieleaks …     0         0      0        0                   0
 7 Itu sekolah ya bukan tem…     0         0      0        0                   0
 8 Karma. I hope it bites K…     0         0      0        0                   0
 9 @stockputout everything …     0         0      0        0                   0
10 Rebecca Black Drops Out …     0         0      0        0                   0
# ℹ 4

In [2]:
install.packages("tm")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [3]:
install.packages("text2vec")


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [4]:
# Load necessary libraries
library(text2vec)
library(dplyr)

# Assuming 'dataset' is your DataFrame and 'processed_tweet' contains preprocessed tweet texts

# If 'processed_tweet' contains strings of list-like text, first remove unwanted characters
processed_tweet <- gsub("[\\[\\]',]", "", dataset$processed_tweet)

# Tokenize the processed tweets
it <- itoken(processed_tweet,
             preprocessor = tolower,
             tokenizer = word_tokenizer,
             progressbar = FALSE)

# Create a vocabulary and prune it to have a maximum of 1000 terms
vocab <- create_vocabulary(it) %>%
  prune_vocabulary(term_count_min = 1,
                   doc_proportion_max = 0.5,
                   vocab_term_max = 1000)

# Vectorize the text
vectorizer <- vocab_vectorizer(vocab)
dtm <- create_dtm(it, vectorizer)

# Convert the Document-Term Matrix (DTM) to a DataFrame
tfidf <- TfIdf$new()
tfidf_matrix <- fit_transform(dtm, tfidf)
tfidf_df <- as.data.frame(as.matrix(tfidf_matrix))

# Display the first few rows of the TF-IDF DataFrame
head(tfidf_df)



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




,bully,school,fuck,like,girl,nigger,joke,dumb,high,people,⋯,minority,plus,racial,writing,afraid,ah,alive,decided,ex,fall
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0.386187,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0.000000,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
# Load necessary libraries
library(stats)
library(ggplot2)

# Assuming 'tfidf_df' is your DataFrame containing the TF-IDF features

# Determine the optimal number of clusters using the elbow method
inertia <- numeric()
K <- 1:10
set.seed(42)  # For reproducibility

for (k in K) {
  kmeans_model <- kmeans(tfidf_df, centers = k, nstart = 5)
  inertia <- c(inertia, kmeans_model$tot.withinss)
}

# Plot the elbow curve
elbow_plot <- data.frame(K = K, Inertia = inertia)
options(repr.plot.width=10, repr.plot.height=6)
ggplot(elbow_plot, aes(x = K, y = Inertia)) +
  geom_point() +
  geom_line() +
  xlab('Number of clusters') +
  ylab('Inertia') +
  ggtitle('Elbow Method For Optimal k') +
  theme_minimal()


In [ ]:
# Load necessary libraries
library(stats)
library(dplyr)

# Assuming 'tfidf_df' is your DataFrame containing the TF-IDF features

# Perform K-Means clustering with the optimal number of clusters (e.g., 4 clusters)
set.seed(42)  # For reproducibility
kmeans_result <- kmeans(tfidf_df, centers = 4, nstart = 5)

# Add the cluster labels to the original dataset
dataset$cluster <- kmeans_result$cluster

# Show the first few rows of the dataset with the cluster labels
head(dataset)


In [ ]:
# Load necessary libraries
library(ggplot2)
library(stats)

# Assuming 'tfidf_df' is your DataFrame containing the TF-IDF features
# and 'dataset' has the cluster labels

# Reduce the dimensions to 2D using PCA
pca_result <- prcomp(tfidf_df, center = TRUE, scale. = TRUE)
X_pca <- data.frame(pca_result$x[, 1:2])

# Add the cluster labels to the PCA result for plotting
X_pca$cluster <- as.factor(dataset$cluster)

# Plot the clusters
options(repr.plot.width=12, repr.plot.height=8)
ggplot(X_pca, aes(x = PC1, y = PC2, color = cluster)) +
  geom_point(size = 2) +
  labs(title = "Clusters Visualization", x = "PCA Component 1", y = "PCA Component 2") +
  theme_minimal() +
  scale_color_manual(values = c("red", "blue", "green", "purple"))


In [ ]:
# Assuming 'dataset' is your DataFrame and it contains the 'tweet_text' and 'cluster' columns

# Function to print sample tweets from each cluster
print_cluster_samples <- function(cluster_number) {
  cat(sprintf("\nCluster %d Sample Tweets:\n", cluster_number))
  cluster_samples <- dataset %>%
    filter(cluster == cluster_number) %>%
    select(tweet_text) %>%
    sample_n(5, replace = FALSE, set.seed(42))

  for (tweet in cluster_samples$tweet_text) {
    cat(sprintf("- %s\n", tweet))
  }
}

# Print sample tweets from each cluster
for (cluster_num in 1:4) {
  print_cluster_samples(cluster_num)
}


In [ ]:
install.packages("wordcloud")

In [ ]:
# Install necessary packages if not already installed
# install.packages("tm")
# install.packages("wordcloud")
# install.packages("ggplot2")

library(dplyr)
library(tm)
library(wordcloud)
library(ggplot2)

# Assuming 'dataset' is your DataFrame with 'tweet_text' and 'cluster' columns

generate_wordcloud_for_cluster <- function(cluster_number) {
  # Filter the dataset for the given cluster number
  cluster_data <- dataset %>%
    filter(cluster == cluster_number)

  # Create a corpus from the tweets in the cluster
  corpus <- Corpus(VectorSource(cluster_data$tweet_text))

  # Clean the text data
  corpus <- corpus %>%
    tm_map(content_transformer(tolower)) %>%
    tm_map(removePunctuation) %>%
    tm_map(removeNumbers) %>%
    tm_map(removeWords, stopwords("en")) %>%
    tm_map(stripWhitespace)

  # Create a term-document matrix
  tdm <- TermDocumentMatrix(corpus)

  # Convert the term-document matrix into a matrix
  tdm_matrix <- as.matrix(tdm)

  # Get the word frequencies
  word_freqs <- sort(rowSums(tdm_matrix), decreasing = TRUE)

  # Convert to a data frame
  word_freqs_df <- data.frame(word = names(word_freqs), freq = word_freqs)

  # Generate the word cloud
  wordcloud(words = word_freqs_df$word, freq = word_freqs_df$freq,
            min.freq = 2, max.words = 100, random.order = FALSE,
            colors = brewer.pal(8, "Dark2"))
}

# Generate word clouds for clusters 1 to 4
for (cluster_num in 1:4) {
  cat(sprintf("\nGenerating word cloud for Cluster %d\n", cluster_num))
  generate_wordcloud_for_cluster(cluster_num)
}
